In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files 
# under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv("../input/Kannada-MNIST/train.csv")
test_df = pd.read_csv("../input/Kannada-MNIST/test.csv")
train_additional_df = pd.read_csv("../input/Kannada-MNIST/Dig-MNIST.csv")

train_df.head()

In [ ]:
train_df = pd.concat([train_df,train_additional_df], ignore_index=True)

In [ ]:
sns.countplot(train_df["label"])
plt.title("number of example for each label")

## convert pixel values to image

In [ ]:
train_images = train_df.iloc[:,1:].values
print(f"shape of train_images:{train_images.shape}")

train_images = train_images.reshape(-1,28,28,1)
print(f"shape of train_images:{train_images.shape}")

## get labels

In [ ]:
train_labels = train_df.label
train_labels.head()

In [ ]:
plt.figure(figsize=(12,8))
for i in range(1,33):
    plt.subplot(4,8,i)
    randomIndex = np.random.randint(0,train_images.shape[0])
    plt.imshow(train_images[randomIndex,:,:,0],cmap="gray")
    plt.title(train_labels[randomIndex])
    plt.axis("off")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size = 0.1, random_state = 0)

In [ ]:
test_df.head()

In [ ]:
test_images = test_df.iloc[:,1:].values
print(f"shape of test_images:{test_images.shape}")

test_images = test_images.reshape(-1,28,28,1)
print(f"shape of test_images:{test_images.shape}")

test_images = test_images / 255

In [ ]:
test_id = test_df.id
test_id[:5]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
            rescale=1./255
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout,\
                                    Flatten, Dense, BatchNormalization, ZeroPadding2D, LeakyReLU
from tensorflow.keras.regularizers import l2

lamda = 0.0
alpha = 0.1

model = tf.keras.models.Sequential([
    Conv2D(64, (3,3), input_shape=(28, 28, 1), padding="same", kernel_regularizer=l2(lamda)),
    BatchNormalization(),
    LeakyReLU(alpha),
    Conv2D(64, (3,3), padding="same", kernel_regularizer=l2(lamda)),
    BatchNormalization(),
    LeakyReLU(alpha),
    MaxPooling2D(2, 2),
    Dropout(0.2),
    
    Conv2D(128, (3,3), padding="same", kernel_regularizer=l2(lamda)),
    BatchNormalization(),
    LeakyReLU(alpha),
    Conv2D(128, (3,3), padding="same", kernel_regularizer=l2(lamda)),
    BatchNormalization(),
    LeakyReLU(alpha),
    MaxPooling2D(2,2),
    Dropout(0.2),
    
    Conv2D(256, (3,3), padding="same", kernel_regularizer=l2(lamda)),
    BatchNormalization(),
    LeakyReLU(alpha),
    Conv2D(256, (3,3), padding="same", kernel_regularizer=l2(lamda)),
    BatchNormalization(),
    LeakyReLU(alpha),
    MaxPooling2D(2,2),
    Dropout(0.2),
    
    Flatten(),
    Dense(1024, kernel_regularizer=l2(lamda)),
    BatchNormalization(),
    LeakyReLU(alpha),
    Dense(128, kernel_regularizer=l2(lamda)),
    BatchNormalization(),
    LeakyReLU(alpha),
    Dense(10, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer="sgd",
              # categorical_crossentropy is for one-hot vector
              # sparse_categorical_crossentropy is for only label
              loss='sparse_categorical_crossentropy',
              metrics = ['acc'])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                                  verbose=0, mode='auto')

In [ ]:
model_check_point = keras.callbacks.ModelCheckpoint('../input/best_model.hdf5', monitor='val_acc',
                                                       verbose=0, save_best_only=True, mode='auto')

In [ ]:
history = model.fit_generator(
                train_datagen.flow(X_train, y_train, batch_size=128),
                validation_data=validation_datagen.flow(X_val,y_val),
                epochs=50,
                callbacks=[early_stopping,model_check_point],
                verbose=2
)

In [ ]:
i = np.argmin(history.history["val_loss"])
vl = f'{history.history["val_loss"][i]:.3f}'
va = f'{history.history["val_acc"][i] * 100:.3f}'
print(f'val_loss:{vl}')
print(f'val_acc:{va}%')

In [ ]:
import requests

def send_line_notification(message):
    line_token = 'EdfDlQndHEEyffeozYDp6dem5IY8rReixPI51L63bX0'  # 終わったら無効化する
    endpoint = 'https://notify-api.line.me/api/notify'
    message = "\n{}".format(message)
    payload = {'message': message}
    headers = {'Authorization': 'Bearer {}'.format(line_token)}
    requests.post(endpoint, data=payload, headers=headers)

message = f'val_loss:{vl},val_acc:{va}%'
# send_line_notification(message)

In [ ]:
acc      = history.history['acc']
val_acc  = history.history['val_acc']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs   = range(len(acc))


# Plot training and validation accuracy per epoch
plt.plot(epochs,acc)
plt.plot(epochs,val_acc)
plt.ylim(0.7,1.0)
plt.title('Training and validation accuracy')
plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs,loss)
plt.plot(epochs,val_loss)
plt.title('Training and validation loss')

In [ ]:
model = tf.keras.models.load_model('../input/best_model.hdf5')

In [ ]:
y_pred = model.predict_classes(test_images)
y_pred[:10]

In [ ]:
sns.countplot(y_pred)

In [ ]:
submission = pd.DataFrame({
    "id":test_id,
    "label":y_pred
})
submission.head()

In [ ]:
submission.to_csv("submission.csv", index = False)